# Assignment 4 

## Initialization

Import required libraries.

In [ ]:
# Standard ML libaries
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report

# RobertaModel and Tockenizer
from transformers import RobertaTokenizer, RobertaModel


Initialize environment with GPU (or CPU as fallback!).

In [ ]:
# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


device: cpu
random seed: 1234


/home/bloop/anaconda3/envs/cs396/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647329220/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Data Processing

Load in dataset.

In [ ]:
dataset = []

linecount = 0
wordcount = 0

with open("dataset.txt", 'r') as dataset_file:
    while line := dataset_file.readline():
        dataset += line.split()
        linecount += 1
        wordcount += len(line)

print("Loaded " + str(linecount) + " lines and " + str(wordcount) + " words.")


# Minimize dataset for testing,
dataset = dataset[:128]

Loaded 4468825 lines and 300935040 words.


Initialize tokenizer and then prepare the dataset by A) Generating a tokenization and B) Generating a vocabulary list

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base", add_prefix_space = True, clean_up_tokenization_spaces = True)

In [ ]:
print(dataset[:10])
chunk_size = 64
tokens = []

for i in tqdm(range(len(dataset) // chunk_size)):
    sequence_start = i * chunk_size
    sequence_end = (i + 1) * chunk_size
    tokens += tokenizer(dataset[sequence_start:sequence_end], is_split_into_words=True)["input_ids"]
    print(str(sequence_start) + "..." + str(sequence_end))

print("processed tokens : " + str(len(tokens)))

['The', 'White', 'Monkey', 'is', 'a', '1925', 'American', 'silent', 'drama', 'film,']


0...64
64...128
processed tokens : 170


Take tokens and shtick em in the model to get the embeddings?

In [ ]:
model = RobertaModel.from_pretrained('roberta-base')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

output

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1162,  0.1060, -0.0101,  ..., -0.0897, -0.0220, -0.0182],
         [ 0.0594, -0.0531, -0.0033,  ..., -0.2571,  0.0529,  0.4554],
         [ 0.0060,  0.2149,  0.0754,  ...,  0.0744,  0.1305,  0.0811],
         ...,
         [-0.1735,  0.0330,  0.0861,  ...,  0.2404,  0.2087, -0.0735],
         [-0.1080,  0.1013, -0.0331,  ..., -0.1196, -0.0253, -0.0431],
         [ 0.0159,  0.0289,  0.0789,  ...,  0.1065,  0.0235,  0.0473]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.0287,  0.1055,  0.0847, -0.0798, -0.2726,  0.0077, -0.0934, -0.1399,
          0.0172, -0.0689,  0.1391,  0.1346, -0.1971, -0.1770, -0.2654,  0.0451,
         -0.0320, -0.0280,  0.2640, -0.0239, -0.0305,  0.0018,  0.3647,  0.0869,
          0.1374, -0.0802, -0.0496, -0.3941, -0.0076,  0.1177,  0.1581,  0.0949,
         -0.0519, -0.0592,  0.1684,  0.1593,  0.3215, -0.7168,  0.1521,  0.2916,
         -0.2329,  0.0027, -0.30